In [14]:
from API_utils import fetch_from_API, show_dates, show_files_for_site_date
import os
import numpy as np
import json

For now we will just do TEAKs oldest Lidar flight

In [2]:
site = 'TEAK'
productcode = 'DP1.30003.001'
data_path = '/home/jovyan/tmp'

os.makedirs(data_path, exist_ok=True)

#show_dates(site, productcode)


In [3]:
files = show_files_for_site_date(productcode, site, '2018-06')
laz = []
for file in files:
    if 'full_boundary.kml' in file['name']:
        boundaries = file
    if 'classified_point_cloud_colorized.laz' in file['name']:
        laz.append(file)

In [71]:
import urllib
import requests
import hashlib


def download_from_NEON_API(f, data_path):

    attempts = 0 
    while attempts < 4:
        try:
            # get the file 
            handle = requests.get(f['url'])
            
            #check the md5 if it exists
            if f['md5']:
                md5 = hashlib.md5(handle.content).hexdigest()
                if md5 == f['md5']:
                    success = True
                    attempts = 4
                else:
                    fmd5 = f['md5']
                    print(f'md5 mismatch on attempt {attempts}')
                    success = False
                    attempts = attempts + 1
            else: 
                success = True
                attempts = 4
        except Exception as e:
            print(f'Warning:\n{e}')
            success = False
            attempts = attempts + 1
    # write the file
    if success:
        fname = os.path.join(data_path, f['name'])
        with open(fname, 'wb') as sink:
            sink.write(handle.content)
    else:
        raise Exception('failed to download')

We need to isntall pdal to process the laz files

!pip install pdal

Ug, now have to ```conda update numpy``` to use pdal. Need to pin these versions down

In [73]:
import pdal
from string import Template
import subprocess
import time

In [59]:
def make_pipe(f, bbox, out_path, resolution=1):
    '''Creates, validates and then returns the pdal pipeline
    
    Arguments:
    bbox       -- Tuple - Bounding box in srs coordintes (default srs is EPSG:3857),
                  in the form: ([xmin, xmax], [ymin, ymax]).
    outpath   -- String - Path where the CHM shall be saved. Must include .tif exstension.
    srs        -- String - EPSG identifier for srs  being used. Defaults to EPSG:3857
                  because that is what ept files tend to use.
    threads    -- Int - Number os threads to be used by the reader.ept. Defaults to 4.
    resolution -- Int or Float - resolution (m) used by writers.gdal
    '''
    
    t = Template('''
    {
        "pipeline": [
            {
            "filename": "${f}",
            "type": "readers.las",
            "tag": "readdata"
            },
            {
            "type":"filters.outlier",
            "method":"radius",
            "radius":1.0,
            "min_k":4
            },
            {
            "type":"filters.range",
            "limits":"Classification[3:5]"
            },
            {
            "type":"filters.optimalneighborhood",
            "min_k":8,
            "max_k": 50
            },
            {
            "type":"filters.covariancefeatures",
            "knn":10,
            "threads": 2,
            "feature_set": "all"
            },
            {
            "filename": "${outpath}_Anisotropy.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Anisotropy",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_DemantkeVerticality.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "DemantkeVerticality",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_Density.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Density",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_Eigenentropy.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Eigenentropy",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_Linearity.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Linearity",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_Omnivariance.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Omnivariance",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_Planarity.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Planarity",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_Scattering.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Scattering",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_EigenvalueSum.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "EigenvalueSum",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_SurfaceVariation.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "SurfaceVariation",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}_Verticality.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Verticality",
            "bounds": "${bbox}"
            }
        ]
    }''')

    pipe = t.substitute(f=f, bbox=bbox, outpath=out_path, resolution=resolution)
    pipeline = pdal.Pipeline(pipe)
    if pipeline.validate():
        return(pipeline)
    else:
        raise Exception('Bad pipeline (sorry to be so ambigous)!')

In [ ]:


    Anisotropy

    DemantkeVerticality

    Density

    Eigenentropy

    Linearity

    Omnivariance

    Planarity

    Scattering

    EigenvalueSum

    SurfaceVariation

    Verticality


In [81]:
def xxx(f, data_path):
    
    # name of file to be stored
    name = os.path.join(data_path, f['name'])
    size = f['size']
    
    # Download the laz
    download_from_NEON_API(f, data_path)
    
    # wait until the file is fully written
    while not os.path.exists(name):
        time.sleep(1)

    on_disk = 0
    while on_disk < size:
        if os.path.isfile(name):
            on_disk = os.path.getsize(name)
            print(f'{on_disk} v {size}')
            time.sleep(1)
    
    # find the bounds
    cmd = f'pdal info {name}'
    reply = subprocess.run(cmd, shell=True, capture_output=True)
    meta = json.loads(reply.stdout)
    bbox = meta['stats']['bbox']['native']['bbox']
    bounds = ([bbox['minx'], bbox['maxx']], [bbox['miny'], bbox['maxy']])  
    
    # make and execute the pdal pipeline
    pipeline = make_pipe(name, bounds, data_path, resolution=1)
    count = pipeline.execute()
    print(count)

In [6]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

results = []
for f in laz[:2]:
    results.append(delayed(download_from_NEON_API)(f, data_path))

with ProgressBar():
    computed = compute(*results)

[########################################] | 100% Completed |  0.9s


In [82]:
for f in laz[:1]:
    xxx(f, data_path)

167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024
167 v 4024


KeyboardInterrupt: 

In [95]:
name

'/home/jovyan/tmp/NEON_D17_TEAK_DP1_323000_4108000_classified_point_cloud_colorized.laz'

In [98]:
f = laz[3]
download_from_NEON_API(f, data_path)
name = os.path.join(data_path, f['name'])

In [99]:
cmd = f'pdal info {name}'
reply = subprocess.run(cmd, shell=True, capture_output=True)
reply

CompletedProcess(args='pdal info /home/jovyan/tmp/NEON_D17_TEAK_DP1_321000_4093000_classified_point_cloud_colorized.laz', returncode=1, stdout=b'', stderr=b"PDAL: readers.las: File signature is not 'LASF', is this an LAS/LAZ file?\n\n")

In [88]:

reply = subprocess.run(cmd, shell=True, capture_output=True)
meta = json.loads(reply.stdout)
bbox = meta['stats']['bbox']['native']['bbox']
bounds = ([bbox['minx'], bbox['maxx']], [bbox['miny'], bbox['maxy']]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)